# ID2214/FID3214 Assignment 2 Group no. [enter]
### Project members: 
[Ivan Klabucar, klabucar@kth.se]
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time

## Reused functions from Assignment 1

In [27]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def create_column_filter(df):
    df_copy = df.copy()
    cols = [x for x in df.columns if not (x == 'CLASS' or x == 'ID')]
    not_dropped = df_copy.loc[:, cols].dropna(how='all', axis='columns').columns
    df_copy.drop(columns=[c for c in cols if not c in not_dropped], inplace=True)
    for c in df_copy:
        if c == 'CLASS' or c == 'ID': continue
        u_vals = pd.unique(df_copy[c])
        num_of_unique = len(u_vals)
        if pd.isna(u_vals).any(): num_of_unique -= 1
        if num_of_unique == 1: df_copy.drop(columns=c,inplace=True)
    return df_copy, list(df_copy.columns)

def apply_column_filter(df, column_filter):
    new_df = df.copy()
    to_drop = [c for c in new_df.columns if c not in column_filter]
    new_df.drop(columns=to_drop, inplace=True)
    return new_df


def apply_bins(df, binning):
    new_df = df.copy()
    for c in binning:
        num_of_bins = len(binning[c]) - 1
        categories = [i for i in range(num_of_bins)]
        new_df[c] = pd.cut(new_df[c], binning[c], labels=categories)
    return new_df

def create_bins(df, nobins=10, bintype='equal-width'):
    binning = dict()
    for c in df:
        if c in ['CLASS', 'ID']: continue
        if not df[c].dtype in [int, float]: continue
        bins = None
        if bintype == 'equal-width': _, bins = pd.cut(df[c], nobins, duplicates='drop', retbins=True)
        elif bintype == 'equal-size': _, bins = pd.qcut(df[c], nobins, duplicates='drop', retbins=True)
        bins[0] = -np.inf
        bins[-1] = np.inf
        binning[c] = bins
    
    return apply_bins(df, binning), binning

def accuracy(df, correctlabels):
    m = df.idxmax(axis="columns")
    return sum([ 1 if r == label else 0 for r, label in zip(m, correctlabels)]) / len(m)

def brier_score(df, correctlabels):
    nLabels = len(df.columns)
    labelMaps = [ np.zeros(nLabels) for _ in range(len(correctlabels)) ]
    for l, lm in zip(correctlabels, labelMaps):
        lm[np.where(df.columns==l)[0][0]] = 1.0

    ress = []
    for index, row in df.iterrows():
        msq = 0.0
        msq = sum((labelMaps[index] - row) ** 2)
        ress.append(msq)
    return sum(ress) / len(ress)

def auc(df, correctlabels):
    total_auc = 0.0

    for c in df:
        scores = []
        thresholds = list(df[c])
        corrected = [ l == c for l in correctlabels]

        for t in thresholds:
            predictions = list(df[c] >= t)
            tp = sum([ 1 if p and l else 0 for p, l in zip(predictions, corrected) ])
            tn = sum([ 1 if (not p) and (not l) else 0 for p, l in zip(predictions, corrected) ])
            fp = sum([ 1 if p and (not l) else 0 for p, l in zip(predictions, corrected) ])
            fn = sum([ 1 if (not p) and l else 0 for p, l in zip(predictions, corrected) ])

            tpr = tp / (tp + fn)
            fpr = fp / (fp + tn)

            scores.append((t, tpr, fpr))
        scores =  [[0,0,0]] + sorted(scores, key=lambda x: -x[0]) + [[1, 1, 1]]

        auc = 0.0
        lastTPR = 0.0
        lastFPR = 0.0
        for t, tpr, fpr in scores:
            auc += lastTPR * (fpr - lastFPR)
            auc += (tpr - lastTPR) * (fpr - lastFPR) / 2.0
        
            lastTPR = tpr
            lastFPR = fpr
        
        total_auc += auc * (
            len([ l for l in correctlabels if l == c]) / len(correctlabels)
        )

    return total_auc


## 1. Define the class kNN

In [3]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies



In [4]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.01 s.
Testing time (k=1): 0.14 s.
Testing time (k=3): 0.15 s.
Testing time (k=5): 0.15 s.
Testing time (k=7): 0.15 s.
Testing time (k=9): 0.15 s.



'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [5]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [30]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors

class NaiveBayes:
    def __init__(self):
        self.column_filter = None
        self.binning = None
        self.labels = None
        self.class_priors = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None
    
    def fit(self, df, nobins=10, bintype='equal-width'):
        local_df, self.column_filter = create_column_filter(df)
        local_df, self.binning = create_bins(local_df, nobins=nobins, bintype=bintype)
        local_df["CLASS"] = local_df["CLASS"].astype("category")
        self.labels = [x for x in local_df["CLASS"].cat.categories]
        f = local_df.groupby('CLASS').size()
        s = local_df.groupby('CLASS').size().sum()
        self.class_priors = {l: f.loc[l] / s for l in self.labels}
        
        
        self.feature_class_value_counts = dict()
        self.feature_class_counts = dict()
        
        for col in local_df:
            if col in ["CLASS", "ID"]: continue
            
            temp_df = local_df.dropna(how='any', axis='index', subset=[col, 'CLASS'])
            self.feature_class_value_counts[col] = dict(temp_df.groupby([col, 'CLASS']).size())
            grouped = temp_df['CLASS'].value_counts()
            self.feature_class_counts[col] = {l: grouped.loc[l] for l in self.labels}
    
    def predict(self, df):
        def normalize_row(x, priors):
            s = sum(x)
            if s == 0:
                return [priors[p] for p in x.index]
            else:
                return x / s
        
        local_df = apply_column_filter(df, self.column_filter)
        local_df = apply_bins(local_df, self.binning)
        to_keep = self.column_filter
        if 'CLASS' in to_keep: to_keep.remove('CLASS')
        if 'ID' in to_keep: to_keep.remove('ID')
        local_df = local_df.drop(['ID', 'CLASS'], axis=1, errors='ignore')
        local_df = local_df.dropna(how='any', axis='index', subset=to_keep)
        
        result = pd.DataFrame(np.ones((len(local_df), len(self.labels))), columns=self.labels, index=local_df.index)
        
        for clss in self.labels:
            for f in local_df:
                d = self.feature_class_value_counts[f]
                dc = self.feature_class_counts[f]
                prob = local_df.apply(lambda x: d[(x[f], clss)] if (x[f], clss) in d else 0, axis=1) / dc[clss]
                result[clss] = result[clss] * prob
            result[clss] = result[clss] * self.class_priors[clss]
        
        result = result.apply(lambda x: normalize_row(x, self.class_priors), axis=1)
        return result


In [31]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.12 s.
Testing time (3, 'equal-width'): 4.29 s.
Training time (3, 'equal-size'): 0.08 s.
Testing time (3, 'equal-size'): 0.29 s.
Training time (5, 'equal-width'): 0.09 s.
Testing time (5, 'equal-width'): 4.48 s.
Training time (5, 'equal-size'): 0.14 s.
Testing time (5, 'equal-size'): 0.25 s.
Training time (10, 'equal-width'): 0.10 s.
Testing time (10, 'equal-width'): 4.55 s.
Training time (10, 'equal-size'): 0.10 s.
Testing time (10, 'equal-size'): 0.27 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [32]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263


### Comment on assumptions, things that do not work properly, etc.

I'm assuming that no missing values will appear in the test data set.